In [1]:
import numpy as np

In [8]:
np.dot(np.ones((4,8)).T,np.array([-2, 1 ,0.5 ,1]).reshape(-1,1))

array([[0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5],
       [0.5]])

In [ ]:
def calculate_weighted_sum(x, w, b):
    """
    Compute the weighted sum of inputs.

    Parameters:
    x (numpy.ndarray): Input data.
    w (numpy.ndarray): Weights matrix.
    b (numpy.ndarray): Bias vector.

    Returns:
    numpy.ndarray or None: Weighted sum of inputs, or None if an error occurs.
    """
    try:
        w = w.transpose()
        z = np.dot(w, x) + b

        return z
    except Exception as e:
        # Handle exceptions, if any, and print an error message.
        print("An error occurred:", e)
        return


In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid function.

    Parameters:
    z (numpy.ndarray): Input array.

    Returns:
    numpy.ndarray: Output of the sigmoid function.
    """
    try:
       return 1 / (1 + np.exp(-z))
    except Exception as e:
        # Handle exceptions, if any, and print an error message.
        print("An error occurred:", e)
        return

In [ ]:
def sum_of_square(y, t):
    """
    Calculate the sum of squares of differences between two arrays.

    Parameters:
    y (array-like): Predicted values.
    t (array-like): Target or true values.

    Returns:
    float: The sum of squares of differences.
    """
    try:
        # Calculate the sum of squares of differences
        return 0.5 * np.sum((y - t) ** 2)
    except Exception as e:
        # Handle exceptions, if any, and print an error message.
        print("An error occurred:", e)
        return None  # Return None if an error occurs

In [ ]:
def delta_n(a_n, t_n):
    """
    Calculate the error (delta) for an output neuron in a neural network.

    Parameters:
    a_n (float): Activation of the output neuron.
    t_n (float): Target or true value for the output neuron.

    Returns:
    float: The calculated error (delta) for the output neuron.
    """
    try:
        # Calculate the error (delta) using the derivative of the sigmoid function
        return (a_n - t_n) * a_n * (1 - a_n)
    except Exception as e:
        # Handle exceptions, if any, and print an error message.
        print("An error occurred:", e)
        return None  # Return None if an error occurs

In [ ]:
def delta_m(a_m, w, s_n):
    """
    Calculate the error (delta) for a neuron in a hidden layer of a neural network.

    Parameters:
    a_m (float): Activation of the neuron in the hidden layer.
    w (array-like): Weights connecting the hidden layer to the subsequent layer.
    s_n (array-like): Error (delta) at the subsequent layer.

    Returns:
    float: The calculated error (delta) for the neuron in the hidden layer.
    """
    try:
        # Calculate the error (delta) using the dot product of weights and subsequent layer error
        return np.dot(w, s_n) * a_m * (1 - a_m)
    except Exception as e:
        # Handle exceptions, if any, and print an error message.
        print("An error occurred:", e)
        return None  # Return None if an error occurs

In [ ]:
def feedforward(input_x, weights, biases):
    """
    Perform the feedforward process of a neural network.

    Parameters:
    input_x (array-like): Input data to the neural network.
    weights (list of arrays): List of weight matrices for each layer.
    biases (list of arrays): List of bias vectors for each layer.

    Returns:
    activation_values (list of arrays): List of activation values for each layer.
    """

    # Combine weights and biases for each layer
    weights_biases = zip(weights, biases)

    # Initialize lists to store weighted sums and activation values
    weighted_sums = []
    activation_values = []

    # Initialize the list of inputs with the initial input
    inputs = [input_x]

    # Iterate over each layer's weights and biases
    for layer_weights, layer_biases in weights_biases:
        # Calculate the weighted sum for the current layer
        weighted_sum = calculate_weighted_sum(inputs[-1], layer_weights, layer_biases)

        # Apply the sigmoid activation function to the weighted sum
        activation = sigmoid(weighted_sum)

        # Append the activation values to the list
        activation_values.append(activation)

        # Update the inputs list with the activation values for the next layer
        inputs.append(activation)

    return activation_values


In [ ]:
def neural_training(input_x, t):
    """
    Train a neural network using gradient descent.

    Parameters:
    input_x (array-like): Input data to the neural network.
    t (array-like): Target or true values for the neural network.

    Returns:
    list: List of costs (errors) at each iteration of training.
    """
    input_x = input_x.reshape(-1, 1)
    t = t.reshape(-1, 1)

    number_of_layers_excluding_input = 2
    number_of_nodes_per_layer = [4, 8, 3]
    learning_rate = 0.1
    # Initialize weights and biases
    weights = [np.ones([number_of_nodes_per_layer[i], number_of_nodes_per_layer[i + 1]]) for i in range(number_of_layers_excluding_input)]
    biases = [np.ones(num).reshape(-1, 1) for num in number_of_nodes_per_layer[1:]]

    # Initialize the list to store costs
    cost = []

    # Perform the feedforward pass
    output_activations = feedforward(input_x, weights, biases)

    # Calculate and append the initial cost
    cost.append(sum_of_square(output_activations[-1], t))

    # Initialize a list to store errors
    errors = []
    print('for sn a=',output_activations[-1])
    # Calculate the error for the output layer
    s_n = delta_n(output_activations[-1], t)
    errors.append(s_n)


    # Backpropagate the error through the network
    for (a_m, w_mn) in zip(output_activations[:-1][::-1], weights[::-1]):
        s_n = errors[-1]
        print(f'am={a_m}')

        print(f'sn={s_n}')
        print('w',w_mn)
        s_m = delta_m(a_m, w_mn, s_n)
        errors.append(s_m)

    # Prepare inputs for weight and bias update
    inputs = output_activations[:-1]
    inputs.insert(0, input_x)

    # Update weights and biases using gradient descent
    for (i, (a_m, s_n)) in enumerate(zip(inputs, errors[::-1])):
        s_n=s_n.reshape(-1,1).T
        a_m=a_m.reshape(-1,1)
        # if i==0:
        #     print(f'a{i}={a_m}')

        #     print(f's{i}={s_n}')

        weights[i] -= learning_rate * s_n * a_m

        biases_update = (learning_rate * s_n).reshape(-1, 1)
        biases[i] -= biases_update

    # Perform another feedforward pass to calculate the updated cost
    output_activations = feedforward(input_x, weights, biases)
    cost.append(sum_of_square(output_activations[-1], t))

    return cost


In [ ]:
if __name__ == "__main__":
    # Read input and target values
    values = list(map(float, input().split()))
    inputs = np.array(values[:4])
    targets = np.array(values[4:])

    cost=neural_training(inputs,targets)
    for loss in cost:
        print(f'{loss: .4f}')
    #1.4 0 -2.5 -3 0.4 0.6 0
    #-2 1 0.5 -1 0 0 1


-2 1 0.5 -1 0 0 1
for sn a= [[0.9823693]
 [0.9823693]
 [0.9823693]]
am=[[0.37754067]
 [0.37754067]
 [0.37754067]
 [0.37754067]
 [0.37754067]
 [0.37754067]
 [0.37754067]
 [0.37754067]]
sn=[[ 0.0170145 ]
 [ 0.0170145 ]
 [-0.00030536]]
w [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
 0.9652
 0.9647
